In [1]:
import os
import pandapower as pp
import pandapower.networks as pn
import networkx as nx
import numpy as np
import pandas as pd
import warnings
from scipy.stats import mode

# Import our custom Bayesian model classes

from bayesgrid import BayesianPowerModel, BayesianFrequencyModel, BayesianDurationModel, BayesianImpedanceModel

from bayesgrid import create_osm_pandapower_network,save_bus_metric_samples,save_power_phase_samples,save_impedance_samples

from bayesgrid import (
        preprocess_power_and_phase_data, 
        preprocess_frequency_data, 
        preprocess_duration_data, 
        preprocess_impedance_data
    )

# Suppress warnings for a cleaner tutorial
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

print("Libraries imported successfully.")

WARNING (pytensor.configdefaults): g++ not available, if using conda: `conda install gxx`
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.


Helper functions for saving are defined.
Libraries imported successfully.




# 📓 Tutorial: Training New Models with `bayesgrid`

While `bayesgrid` comes with powerful, pre-trained models, its true strength lies in its ability to learn from *your* data. If you have a dataset from a specific utility or region, you can train `bayesgrid` to generate synthetic networks that match *your* local conditions.

This tutorial will guide you through:

1.  Creating a dummy (artificial) dataset that has the required columns.
2.  Using the built-in `preprocess_` functions to prepare your data.
3.  Calling the `.learn()` method for each of the four main model types.
4.  Saving your new, custom-trained model traces.



In [2]:
# --- 1. Create Artificial BUS Data ---
N_BUSES = 10
np.random.seed(42)

# Create normalized hop distance (0.0 to 1.0)
hop_dist = np.random.rand(N_BUSES)

# Create phases
phases = np.random.choice(
    ['A', 'B', 'C', 'AB', 'BC', 'BC', 'ABC'], 
    N_BUSES, 
    p=[0.15, 0.15, 0.15, 0.1, 0.1, 0.1, 0.25] # Biased towards 3-phase
)

# Create power, then zero-out non-existing phases
P_A = np.random.lognormal(4, 1, N_BUSES)
P_B = np.random.lognormal(4, 1, N_BUSES)
P_C = np.random.lognormal(4, 1, N_BUSES)

mask_no_A = np.char.find(phases, 'A') == -1
mask_no_B = np.char.find(phases, 'B') == -1
mask_no_C = np.char.find(phases, 'C') == -1

# Apply the masks
P_A[mask_no_A] = 0
P_B[mask_no_B] = 0
P_C[mask_no_C] = 0

# Create reliability data
# Frequency (count data, use Poisson)
fic = np.random.poisson(lam=1.0 + hop_dist * 3, size=N_BUSES) # Freq increases with distance
# Duration (use Weibull, with ~30% zeros for the "hurdle")
dur = np.array([0 if np.random.rand() < 0.3 else np.random.weibull(1.5) * (2 + hop_dist*5) 
                for hop_dist in hop_dist])

# Assemble the Bus DataFrame
raw_bus_data = pd.DataFrame({
    'bus_id': range(N_BUSES),
    'hop_distance_normalized': hop_dist,
    'phases': phases,
    'P_A': P_A,
    'P_B': P_B,
    'P_C': P_C,
    'FIC_total': fic,
    'duration': dur
})

print("--- Artificial Bus Data ---")
print(raw_bus_data.head())


# --- 2. Create Artificial LINE Data ---
N_LINES = 20
elec_dist = np.random.rand(N_LINES)

# Create R1 and X1 (Gamma is good for positive, skewed data)
R1 = np.random.gamma(2, 0.1 + elec_dist * 0.2, N_LINES)
X1 = np.random.gamma(3, 0.2 + elec_dist * 0.1, N_LINES)

# Assemble the Line DataFrame
raw_line_data = pd.DataFrame({
    'line_id': range(N_LINES),
    'electrical_distance_to_substation': elec_dist,
    'R1': R1,
    'X1': X1
})

print("\n--- Artificial Line Data ---")
print(raw_line_data.head())

--- Artificial Bus Data ---
   bus_id  hop_distance_normalized phases        P_A        P_B        P_C  \
0       0                 0.374540      A  19.829463   0.000000   0.000000   
1       1                 0.950714    ABC  74.757335  79.495309   7.693147   
2       2                 0.731994    ABC  22.020546  29.944968  14.466186   
3       3                 0.598658      B   0.000000  40.784669   0.000000   
4       4                 0.156019      B   0.000000  29.913007   0.000000   

   FIC_total  duration  
0          2  0.000000  
1          3  0.000000  
2          3  2.955830  
3          4  8.904145  
4          1  0.000000  

--- Artificial Line Data ---
   line_id  electrical_distance_to_substation        R1        X1
0        0                           0.929698  0.360919  0.463249
1        1                           0.808120  0.217322  0.610642
2        2                           0.633404  1.242241  1.113390
3        3                           0.871461  0.521802  0.

## Checking the synthetic data

In [3]:
raw_line_data.head()

,line_id,electrical_distance_to_substation,R1,X1
0,0,0.929698,0.360919,0.463249
1,1,0.808120,0.217322,0.610642
2,2,0.633404,1.242241,1.113390
3,3,0.871461,0.521802,0.321428
4,4,0.803672,0.055825,0.226756


In [5]:
raw_bus_data.head()

,bus_id,hop_distance_normalized,phases,P_A,P_B,P_C,FIC_total,duration
0,0,0.374540,A,19.829463,0.000000,0.000000,2,0.000000
1,1,0.950714,ABC,74.757335,79.495309,7.693147,3,0.000000
2,2,0.731994,ABC,22.020546,29.944968,14.466186,3,2.955830
3,3,0.598658,B,0.000000,40.784669,0.000000,4,8.904145
4,4,0.156019,B,0.000000,29.913007,0.000000,1,0.000000


## Section 1: Learning the Impedance Model (R1 & X1)

This model learns the Gamma Mixture distribution for `R1` and `X1` based on the `electrical_distance_to_substation`.

In [9]:
n_zones_impd

9

In [ ]:
print("--- Learning new Impedance Model ---")

# 1. Preprocess the data
# We'll use 10 discrete zones for impedance
(elec_zones, r1_vals, x1_vals, n_zones_impd) = preprocess_impedance_data(
    raw_line_data, 
    n_discrete=9
)

# 2. Initialize an empty model
# We pass trace_path=None to signal that we are in "learn mode"
# and don't want to download the default traces.
bim_new = BayesianImpedanceModel(trace_r_path=None, trace_x_path=None)

# 3. Learn R1
# We use small draws/tune for a fast tutorial.
# For a real model, use 1000-2000.
print("\nLearning R1 model...")
trace_r = bim_new.learn_r(
    elec_dist_idx=elec_zones,
    r1_data=r1_vals,
    n_zones=n_zones_impd,
    draws=500, tune=500, cores=1
)

# 4. Learn X1
print("\nLearning X1 model...")
trace_x = bim_new.learn_x(
    elec_dist_idx=elec_zones,
    x1_data=x1_vals,
    n_zones=n_zones_impd,
    draws=500, tune=500, cores=1
)


# 5. Save the new traces
os.makedirs("my_new_traces", exist_ok=True)
bim_new.save_trace(trace_r, 'my_new_traces/my_r_trace.nc')
bim_new.save_trace(trace_x, 'my_new_traces/my_x_trace.nc')

print("\nImpedance models learned and saved!")


--- Learning new Impedance Model ---
Preprocessing impedance data...
Preprocessing complete. Found 20 valid lines across 9 zones.
Fetching default R-model trace...
Fetching default X-model trace...
Loading R1 trace from C:\Users\hoc\AppData\Local\bayesgrid\bayesgrid\Cache\trace_r.nc...
R1 model loaded. Trained with 10353 lines and 10 zones.
Loading X1 trace from C:\Users\hoc\AppData\Local\bayesgrid\bayesgrid\Cache\trace_x.nc...
X1 model loaded. Trained with 10353 lines and 10 zones.

Learning R1 model...
Building r1_likelihood model for learning with 20 lines...
Starting sampling with args: {'draws': 500, 'tune': 500, 'cores': 1}


Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [mean_1, mean_2_offset, mean_3_offset, cv, w_by_zone]


Sampling 2 chains for 500 tune and 500 draw iterations (1_000 + 1_000 draws total) took 246 seconds.
There were 6 divergences after tuning. Increase `target_accept` or reparameterize.
We recommend running at least 4 chains for robust computation of convergence diagnostics


Learning complete.

Learning X1 model...
Building x1_likelihood model for learning with 20 lines...
Starting sampling with args: {'draws': 500, 'tune': 500, 'cores': 1}


Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [mean_1, mean_2_offset, mean_3_offset, cv, w_by_zone]


Sampling 2 chains for 500 tune and 500 draw iterations (1_000 + 1_000 draws total) took 283 seconds.
There were 20 divergences after tuning. Increase `target_accept` or reparameterize.
We recommend running at least 4 chains for robust computation of convergence diagnostics


Learning complete.
Trace saved to my_new_traces/my_r_trace.pickle
Trace saved to my_new_traces/my_x_trace.pickle

Impedance models learned and saved!


You can now use the new learned trace file to create a synthetic grid. 

For example



## Section 2: Learning the Failure Frequency Model (CAIFI/FIC)

This model learns the Negative Binomial distribution for `FIC_total` (failure counts) based on the `hop_distance_normalized`.

In [14]:
print("--- Learning new Failure Frequency Model ---")

# 1. Preprocess the data
# We'll use 30 discrete zones for reliability
(hop_zones_freq, freq_data, n_zones_freq) = preprocess_frequency_data(
    raw_bus_data, 
    n_discrete=30, 
    frequency_col='FIC_total'
)

# 2. Initialize an empty model
bfm_new = BayesianFrequencyModel(trace_path=None)

# 3. Learn the model
print("\nLearning Frequency (FIC) model...")
trace_freq = bfm_new.learn(
    hop_zone_idx=hop_zones_freq,
    frequency_data=freq_data,
    n_zones=n_zones_freq,
    draws=500, tune=500, cores=1
)

# 4. Save the new trace
bfm_new.save_trace('my_new_traces/my_freq_trace.nc')

print("\nFrequency model learned and saved!")

--- Learning new Failure Frequency Model ---
Preprocessing frequency data...
Preprocessing complete. Found 10 buses across 8 zones.
No trace_path provided. Fetching default pre-trained model...
Loading trace from C:\Users\hoc\AppData\Local\bayesgrid\bayesgrid\Cache\trace_fic.nc...
Successfully loaded pre-trained model.
Model was trained with 9328 buses and 30 zones.

Learning Frequency (FIC) model...
Building model for learning with 10 buses and 8 zones...
Starting sampling with args: {'draws': 500, 'tune': 500, 'cores': 1}


Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [mu_by_zone, alpha_dispersion]


Sampling 2 chains for 500 tune and 500 draw iterations (1_000 + 1_000 draws total) took 55 seconds.
We recommend running at least 4 chains for robust computation of convergence diagnostics


Learning complete. Trace is stored in self.trace.
Trace saved to my_new_traces/my_freq_trace.nc

Frequency model learned and saved!


## Section 3: Learning the Failure Duration Model (CAIDI/DIC)

This model learns the Hurdle-Weibull distribution for `duration` (failure duration) based on the `hop_distance_normalized`.

In [15]:
print("--- Learning new Failure Duration Model ---")

# 1. Preprocess the data
# This function handes the "hurdle" logic automatically
(hop_all, is_pos, hop_pos, pos_durs, n_zones_dur) = preprocess_duration_data(
    raw_bus_data, 
    n_discrete=30, 
    duration_col='duration'
)

# 2. Initialize an empty model
bdm_new = BayesianDurationModel(trace_path=None)

# 3. Learn the model
print("\nLearning Duration (DIC) model...")
trace_dur = bdm_new.learn(
    hop_zone_idx_all=hop_all,
    is_positive_all=is_pos,
    hop_zone_idx_positive=hop_pos,
    positive_durations=pos_durs,
    n_zones=n_zones_dur,
    draws=500, tune=500, cores=1
)

# 4. Save the new trace
bdm_new.save_trace('my_new_traces/my_dur_trace.nc')

print("\nDuration model learned and saved!")

--- Learning new Failure Duration Model ---
Preprocessing duration data...
Preprocessing complete. Found 10 total buses.
Found 3 buses with positive duration.
No trace_path provided. Fetching default pre-trained model...
Loading trace from C:\Users\hoc\AppData\Local\bayesgrid\bayesgrid\Cache\trace_dic.nc...
Successfully loaded pre-trained model.
Model was trained with 13547 total buses
(8299 positive) and 30 zones.

Learning Duration (DIC) model...
Building model for learning with 10 total buses
(3 positive) and 8 zones...
Starting sampling with args: {'draws': 500, 'tune': 500, 'cores': 1}


Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [p_positive_by_zone, alpha_weibull_by_zone, beta_weibull_by_zone]


Sampling 2 chains for 500 tune and 500 draw iterations (1_000 + 1_000 draws total) took 75 seconds.
There were 9 divergences after tuning. Increase `target_accept` or reparameterize.
We recommend running at least 4 chains for robust computation of convergence diagnostics
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details


Learning complete. Trace is stored in self.trace.
Trace saved to my_new_traces/my_dur_trace.nc

Duration model learned and saved!


## Section 4: Learning the Power & Phase Model

This model learns the complex, graph-aware relationship between `phases`, `P_A`, `P_B`, `P_C`, and `hop_distance_normalized`.


In [18]:

def preprocess_power_and_phase_data(df, n_discrete=10, 
                    hop_col='hop_distance_normalized', 
                    phase_col='phases', 
                    power_cols=['P_A', 'P_B', 'P_C']):
    """
    Preprocesses a raw DataFrame for the BayesianPowerModel.
    ... (code from previous step, unchanged) ...
    """
    print("Preprocessing data...")
    if hop_col not in df.columns:
        raise ValueError(f"Missing required column: {hop_col}")
    if phase_col not in df.columns:
        raise ValueError(f"Missing required column: {phase_col}")

    df_proc = df.copy()

    # --- 1. Discretize Hop Distance ---
    df_proc['hop_distance_discrete'] = pd.cut(
        df_proc[hop_col], 
        bins=n_discrete, 
        labels=False, # Use integer labels 0 to n_discrete-1
        include_lowest=True,
        duplicates='drop'
    )
    
    if df_proc['hop_distance_discrete'].isnull().any():
        warnings.warn("NaNs found in 'hop_distance_discrete' after binning. Filling with zone 0.")
        df_proc['hop_distance_discrete'] = df_proc['hop_distance_discrete'].fillna(0)

    df_proc['hop_zone_idx'] = pd.Categorical(df_proc['hop_distance_discrete']).codes
    hop_zone_idx = df_proc['hop_zone_idx'].values.astype(int)
    n_zones = len(np.unique(hop_zone_idx))

    # --- 2. Encode Phases ---
    phase_map = {'A': 0, 'B': 1, 'C': 2, 'AB': 3, 'AC': 4, 'BC': 5, 'ABC': 6}
    

    phase_category_idx = df_proc[phase_col].map(phase_map).fillna(-1).astype(int)
    
    # --- 3. Filter out invalid data ---
    valid_mask = (phase_category_idx != -1)
    if not valid_mask.all():
        n_dropped = (~valid_mask).sum()
        warnings.warn(f"Dropping {n_dropped} buses with unknown phase categories.")
        df_proc = df_proc[valid_mask]
        hop_zone_idx = hop_zone_idx[valid_mask]
        phase_category_idx = phase_category_idx[valid_mask]

    # --- 4. Get Power Array ---
    if all(col in df_proc.columns for col in power_cols):
         power_array = df_proc[power_cols].values
         if np.isnan(power_array).any():
             n_nans = np.isnan(power_array).sum()
             warnings.warn(f"Found {n_nans} NaNs in power data. Replacing with 0.0.")
             power_array = np.nan_to_num(power_array, nan=0.0)
    else:
        warnings.warn(f"Power columns {power_cols} not found. Returning None for power array.")
        power_array = None

    print(f"Preprocessing complete. {len(hop_zone_idx)} valid buses found across {n_zones} zones.")
    return hop_zone_idx, phase_category_idx, power_array, n_zones

In [29]:
print("--- Learning new Power and Phase Model ---")

# 1. Preprocess the data
# This function automatically handles phase encoding and power values
(hop_zones_power, phases_idx, power_vals, n_zones_power) = preprocess_power_and_phase_data(
    raw_bus_data, 
    n_discrete=10
)

# 2. Initialize an empty model
bhm_new = BayesianPowerModel(trace_path=None)

# 3. Learn the model
# This model is the most complex and may take longer
print("\nLearning Power and Phase model...")
trace_power = bhm_new.learn(
    hop_zone_idx=hop_zones_power,
    observed_phases=phases_idx,
    observed_power_p=power_vals,
    n_zones=10,
    draws=500, tune=500, cores=1
)

# 4. Save the new trace
bhm_new.save_trace('my_new_traces/my_power_trace.nc')

print("\nPower & Phase model learned and saved!")

--- Learning new Power and Phase Model ---
Preprocessing data...
Preprocessing complete. 10 valid buses found across 6 zones.
No trace_path provided. Fetching default pre-trained model...
Loading trace from C:\Users\hoc\AppData\Local\bayesgrid\bayesgrid\Cache\trace_power_and_phase.nc...
Successfully loaded pre-trained model.
Model was trained with 3599 buses and 10 zones.

Learning Power and Phase model...
Building model for learning with 10 buses and 10 zones...
Starting sampling with args: {'draws': 500, 'tune': 500, 'cores': 1}


Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [a_by_zone, probs, alpha_mono, beta_mono, P_potential_mono, alpha_bi, beta_bi, P_potential_bi_total, split_factor_bi, alpha_tri, beta_tri, P_potential_tri_total, split_factor_tri, sigma_p]


c:\Users\hoc\anaconda3\envs\bhmenv\Lib\site-packages\numpy\core\fromnumeric.py:88: RuntimeWarning: overflow 
encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)

c:\Users\hoc\anaconda3\envs\bhmenv\Lib\site-packages\pytensor\tensor\elemwise.py:710: RuntimeWarning: overflow 
encountered in impl (vectorized)
  variables = ufunc(*ufunc_args, **ufunc_kwargs)

c:\Users\hoc\anaconda3\envs\bhmenv\Lib\site-packages\pytensor\tensor\elemwise.py:710: RuntimeWarning: overflow 
encountered in exp
  variables = ufunc(*ufunc_args, **ufunc_kwargs)

Sampling 2 chains for 500 tune and 500 draw iterations (1_000 + 1_000 draws total) took 1897 seconds.
We recommend running at least 4 chains for robust computation of convergence diagnostics
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details


Learning complete. Trace is stored in self.trace.
Trace saved to my_new_traces/my_power_trace.nc

Power & Phase model learned and saved!



## Tutorial Complete!

You have successfully trained a full set of custom Bayesian models on your own (artificial) dataset.

You now have a folder `my_new_traces/` containing:

  * `my_r_trace.nc`
  * `my_x_trace.nc`
  * `my_freq_trace.nc`
  * `my_dur_trace.nc`
  * `my_power_trace.nc`

### How to use your new models:

In your other notebooks (like the `pandapower` or `OSM` tutorials), simply point the model's `__init__` constructor to your new file paths. The generation code will use your custom-trained models instead of the downloaded defaults.

**Example:**

```python
# Instead of this:
# bhm = BayesianPowerModel()

# Do this:
bhm = BayesianPowerModel(trace_path='my_new_traces/my_power_trace.nc')

# ...and the same for the other models...
bfm = BayesianFrequencyModel(trace_path='my_new_traces/my_freq_trace.nc')
bdm = BayesianDurationModel(trace_path='my_new_traces/my_dur_trace.nc')
bim = BayesianImpedanceModel(
    trace_r_path='my_new_traces/my_r_trace.nc',
    trace_x_path='my_new_traces/my_x_trace.nc'
)

# Now, all generation code will use YOUR trained models!
```